## USE GPU

In [1]:
!pip install transformers
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Loading Mutilingual BERT





In [3]:
# Install SentencePiece
!pip install sentencepiece

# Now you can import the tokenizer and model
from transformers import XLMRobertaTokenizer, XLMRobertaModel

# Load tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
model = XLMRobertaModel.from_pretrained("xlm-roberta-base")


In [4]:
def get_bert_embeddings(text):
    # Ensure text is a string
    text = str(text)
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1)

# Assume text1 and text2 are your text data in Spanish or English
text1 = "This is a sample text in English."
text2 = "Este es un texto de ejemplo en español."

# Get BERT embeddings
embedding1 = get_bert_embeddings(text1)
embedding2 = get_bert_embeddings(text2)


In [5]:
# Convert torch tensors to numpy arrays for use with sklearn
embedding1_np = embedding1.numpy()
embedding2_np = embedding2.numpy()

# Compute cosine similarity
similarity = cosine_similarity(embedding1_np, embedding2_np)
print(f'Cosine Similarity: {similarity[0][0]}')
print(similarity)

Cosine Similarity: 0.9966443181037903
[[0.9966443]]


Let's Start Building

##Getting the Data

In [6]:
import zipfile
!wget https://www.dropbox.com/s/yle22nlwxhgf7pb/color.zip?dl=0&file_subpath=%2Fcolor%2Fexperiment1-dataset-color-of-concrete-objects.txt
# Specify the name of the ZIP file
zip_file = './color.zip?dl=0'

# Specify the directory where you want to extract the contents
extract_path = './'

# Create the directory if it doesn't exist
import os
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Contents of {zip_file} have been extracted to {extract_path}')
%cd color



--2023-11-08 12:08:28--  https://www.dropbox.com/s/yle22nlwxhgf7pb/color.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/yle22nlwxhgf7pb/color.zip [following]
--2023-11-08 12:08:28--  https://www.dropbox.com/s/raw/yle22nlwxhgf7pb/color.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc69c482eec3775fbae85ad8dd70.dl.dropboxusercontent.com/cd/0/inline/CHJeoDiHgcLVjZ_iIAy9kKDQbBv12TOrFVwMZ-sMNXdNVdrvwszXNWsQCtY4oN8ezZceGvoCRPU-tJWdUhkLpb5mqvlRijxmoyalcB1mXauxSq3pBOyNDEl24Ydc7IqjZVk/file# [following]
--2023-11-08 12:08:28--  https://uc69c482eec3775fbae85ad8dd70.dl.dropboxusercontent.com/cd/0/inline/CHJeoDiHgcLVjZ_iIAy9kKDQbBv12TOrFVwMZ-sMNXdNVdrvwszXNWsQCtY4oN8ezZceGvoCRPU-tJWdUhkLpb5mqvlRijxmoyalcB1mXauxSq3pBOyNDEl24Ydc7Iq

In [7]:
import pandas as pd
df = pd.read_csv('experiment1-dataset-color-of-concrete-objects.txt',sep='\t', names =['noun','color'])
bless = df

In [ ]:
# Add true_similarity column
def compute_cosine_similarity(noun, color):
    noun_vector = get_bert_embeddings(noun)
    color_vector = get_bert_embeddings(color)
    noun_vector_np = noun_vector.numpy()
    color_vector_np = color_vector.numpy()
    similarity = cosine_similarity(noun_vector_np, color_vector_np)
    return similarity[0][0]

bless['true_similarity'] = bless.apply(lambda row: compute_cosine_similarity(row['noun'], row['color']), axis=1)

color_list = ['black','blue','brown','green','grey','orange','pink','purple','red','white','yellow']

# For each color, compute the cosine similarity and store in a new column
for color in color_list:
    bless[color] = bless['noun'].apply(lambda noun: compute_cosine_similarity(noun, color))

bless['Max_similarity'] = bless[color_list].max(axis=1)
bless['Max_color'] = bless[color_list].idxmax(axis=1)
bless['accuracy'] = (bless['color'] == bless['Max_color']).astype(int)
bless

,noun,color,true_similarity,black,blue,brown,green,grey,orange,pink,purple,red,white,yellow,Max_similarity,Max_color,accuracy
0,ash,grey,0.995708,0.993807,0.992350,0.992676,0.992249,0.995708,0.995764,0.992678,0.994604,0.991192,0.992725,0.993157,0.995764,orange,0
1,banana,yellow,0.998660,0.998606,0.998587,0.998975,0.998396,0.995792,0.995467,0.998550,0.994765,0.997286,0.998637,0.998660,0.998975,brown,0
2,beaver,brown,0.993129,0.993780,0.992721,0.993129,0.992028,0.996579,0.996910,0.993064,0.995602,0.991682,0.992846,0.993360,0.996910,orange,0
3,blood,red,0.997013,0.998621,0.998537,0.998598,0.998488,0.996504,0.996252,0.998562,0.995563,0.997013,0.998346,0.998618,0.998621,black,0
4,broccoli,green,0.990585,0.992589,0.991459,0.992256,0.990585,0.996720,0.997180,0.991813,0.996377,0.992172,0.991962,0.992078,0.997180,orange,0
5,cardboard,brown,0.996009,0.996076,0.995530,0.996009,0.994733,0.997092,0.997201,0.995673,0.996345,0.994958,0.995825,0.995969,0.997201,orange,0
6,carrot,orange,0.997954,0.996880,0.996734,0.996985,0.995993,0.997906,0.997954,0.996813,0.997284,0.995718,0.996242,0.996580,0.997954,orange,1
7,cauliflower,white,0.991812,0.992308,0.991506,0.992359,0.990396,0.996170,0.996385,0.991680,0.996196,0.991606,0.991812,0.991992,0.996385,orange,0
8,cello,brown,0.993247,0.992456,0.992270,0.993247,0.991026,0.996211,0.996817,0.992562,0.996154,0.995855,0.992436,0.992608,0.996817,orange,0
9,chalk,white,0.996092,0.996477,0.996048,0.996556,0.995483,0.997746,0.997662,0.996359,0.997550,0.995400,0.996092,0.996432,0.997746,grey,0


In [ ]:
overall_accuracy = bless['accuracy'].sum() / len(bless)

print(f"Overall Accuracy: {overall_accuracy:.2%}")

Overall Accuracy: 3.85%


In [ ]:
# For each row, rank the cosine similarities and determine the rank of the true color
ranks = []
for index, row in bless.iterrows():
    ranked_colors = row[color_list].sort_values(ascending=False).index.tolist()
    rank_of_true_color = ranked_colors.index(row['color']) + 1
    ranks.append(rank_of_true_color)

# Add ranks to the dataframe
bless['rank'] = ranks

# Calculate median rank
median_rank = pd.Series(ranks).median()

print("Median Rank:", median_rank)

Median Rank: 6.0


In [ ]:
!pip install deep_translator
from deep_translator import GoogleTranslator
to_translate = 'I want to translate this text'
def translate_text(text):
  return GoogleTranslator(source='auto', target='es').translate(text)
# outpout -> Ich möchte diesen Text übersetzen
#from googletrans import Translator

def translate_column_to_spanish(df, column_name1, column_name2):
    # Initialize the translator


    # Translate the words in the specified column to Spanish
    df['nombre_esp'] = df[column_name1].apply(lambda text: translate_text(text,))
    df['color_esp'] = df[column_name2].apply(lambda text: translate_text(text,))

    return df


# Call the translate_column_to_spanish function
translated_df = translate_column_to_spanish(df, 'noun', 'color')


# Display the translated DataFrame
print(translated_df)
bless = translated_df




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00
           noun   color  true_similarity     black      blue     brown  \
0           ash    grey         0.995708  0.993807  0.992350  0.992676   
1        banana  yellow         0.998660  0.998606  0.998587  0.998975   
2        beaver   brown         0.993129  0.993780  0.992721  0.993129   
3         blood     red         0.997013  0.998621  0.998537  0.998598   
4      broccoli   green         0.990585  0.992589  0.991459  0.992256   
5     cardboard   brown         0.996009  0.996076  0.995530  0.996009   
6        carrot  orange         0.997954  0.996880  0.996734  0.996985   
7   cauliflower   white         0.991812  0.992308  0.991506  0.992359   
8         cello   brown         0.993247  0.992456  0.992270  0.993247   
9         chalk   white         0.996092  0.996477  0.996048  0.996556   
10       cherry     red         0.996094  0.996678  0.996743  0.997027   
11        cloud   white         

In [ ]:
print(bless['nombre_esp'].isnull().sum())
print(bless['color_esp'].isnull().sum())

0
0


In [ ]:
embedding1 = get_bert_embeddings(text1)
embedding2 = get_bert_embeddings(text2)

def compute_cosine_similarity(noun, color):
    noun_vector = get_bert_embeddings(noun)
    color_vector = get_bert_embeddings(color)
    noun_vector_np = noun_vector.numpy()
    color_vector_np = color_vector.numpy()
    similarity = cosine_similarity(noun_vector_np, color_vector_np)
    return similarity[0][0]

# Add true_similarity column
bless['true_similarity'] = bless.apply(lambda row: compute_cosine_similarity(row['nombre_esp'], row['color_esp']), axis=1)

color_list = ['gris', 'amarillo', 'marrón', 'rojo', 'verde', 'naranja', 'blanco', 'negro', 'rosa', 'azul']

# For each color, compute the cosine similarity and store in a new column
for color in color_list:
    bless[color] = bless['nombre_esp'].apply(lambda noun: compute_cosine_similarity(noun, color))

bless['Max_similarity'] = bless[color_list].max(axis=1)
bless['Max_color'] = bless[color_list].idxmax(axis=1)
bless['accuracy'] = (bless['color_esp'] == bless['Max_color']).astype(int)
bless

,noun,color,true_similarity,black,blue,brown,green,grey,orange,pink,...,gris,amarillo,marrón,rojo,verde,naranja,blanco,negro,rosa,azul
0,ash,grey,0.997621,0.993807,0.992350,0.992676,0.992249,0.995708,0.995764,0.992678,...,0.997621,0.997504,0.998359,0.998549,0.996449,0.997620,0.995867,0.996394,0.997019,0.996161
1,banana,yellow,0.994150,0.998606,0.998587,0.998975,0.998396,0.995792,0.995467,0.998550,...,0.997764,0.994150,0.995996,0.995441,0.998740,0.994164,0.998646,0.998737,0.998535,0.998631
2,beaver,brown,0.998628,0.993780,0.992721,0.993129,0.992028,0.996579,0.996910,0.993064,...,0.997343,0.997797,0.998628,0.998560,0.996449,0.997465,0.996069,0.996396,0.996793,0.996285
3,blood,red,0.996061,0.998621,0.998537,0.998598,0.998488,0.996504,0.996252,0.998562,...,0.997944,0.994363,0.996282,0.996061,0.998913,0.994643,0.998887,0.998885,0.998850,0.999099
4,broccoli,green,0.989584,0.992589,0.991459,0.992256,0.990585,0.996720,0.997180,0.991813,...,0.991144,0.995772,0.995108,0.995313,0.989584,0.994853,0.989347,0.990199,0.990189,0.989468
5,cardboard,brown,0.995262,0.996076,0.995530,0.996009,0.994733,0.997092,0.997201,0.995673,...,0.991200,0.995392,0.995262,0.995384,0.990672,0.995742,0.991084,0.991253,0.990979,0.991300
6,carrot,orange,0.997140,0.996880,0.996734,0.996985,0.995993,0.997906,0.997954,0.996813,...,0.991037,0.995801,0.995263,0.995423,0.989790,0.997140,0.989611,0.990196,0.989838,0.989691
7,cauliflower,white,0.992131,0.992308,0.991506,0.992359,0.990396,0.996170,0.996385,0.991680,...,0.994130,0.997038,0.996947,0.997276,0.992536,0.996930,0.992131,0.992754,0.993094,0.992511
8,cello,brown,0.995457,0.992456,0.992270,0.993247,0.991026,0.996211,0.996817,0.992562,...,0.989758,0.995575,0.995457,0.995252,0.989285,0.994999,0.989511,0.989891,0.989630,0.989684
9,chalk,white,0.993113,0.996477,0.996048,0.996556,0.995483,0.997746,0.997662,0.996359,...,0.993312,0.996321,0.997140,0.996489,0.993029,0.996313,0.993113,0.993300,0.993483,0.993173


In [ ]:
overall_accuracy = bless['accuracy'].sum() / len(bless)

print(f"Overall Accuracy: {overall_accuracy:.2%}")

Overall Accuracy: 17.31%


In [ ]:
# For each row, rank the cosine similarities and determine the rank of the true color
ranks = []
for index, row in bless.iterrows():
    ranked_colors = row[color_list].sort_values(ascending=False).index.tolist()
    rank_of_true_color = ranked_colors.index(row['color_esp']) + 1
    ranks.append(rank_of_true_color)

# Add ranks to the dataframe
bless['rank'] = ranks

# Calculate median rank
median_rank = pd.Series(ranks).median()

print("Median Rank:", median_rank)

Median Rank: 6.5
